# Keras LSTM Text Generation
Text generation is a important nlp problem which can enable computers to write.

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/TannerGilbert/Tutorials/blob/master/Keras-Tutorials/4.%20LSTM%20Text%20Generation/Keras%20LSTM%20Text%20Generation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/TannerGilbert/Tutorials/blob/master/Keras-Tutorials/4.%20LSTM%20Text%20Generation/Keras%20LSTM%20Text%20Generation.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io

In [ ]:
!wget -O sherlock_homes.txt http://www.gutenberg.org/files/1661/1661-0.txt

--2021-03-24 07:32:09--  http://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607792 (594K) [text/plain]
Saving to: ‘sherlock_homes.txt’

sherlock_homes.txt  100%[===================>] 593.55K  3.40MB/s    in 0.2s    

2021-03-24 07:32:09 (3.40 MB/s) - ‘sherlock_homes.txt’ saved [607792/607792]



In [ ]:
text = open('sherlock_homes.txt', 'r').read().lower()
print('text length', len(text))

text length 581889


In [ ]:
print(text[:1000])

﻿
project gutenberg's the adventures of sherlock holmes, by arthur conan doyle

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.org


title: the adventures of sherlock holmes

author: arthur conan doyle

release date: november 29, 2002 [ebook #1661]
last updated: may 20, 2019

language: english

character set encoding: utf-8

*** start of this project gutenberg ebook the adventures of sherlock holmes ***



produced by an anonymous project gutenberg volunteer and jose menendez



cover



the adventures of sherlock holmes



by arthur conan doyle



contents


   i.     a scandal in bohemia
   ii.    the red-headed league
   iii.   a case of identity
   iv.    the boscombe valley mystery
   v.     the five orange pips
   vi.    the man with the twisted lip
   vii.   the adventure of the blue 

## Map chars to integers

In [ ]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

total chars:  73


In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

## Split up into subsequences

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 193950


In [ ]:
print(sentences[:3])
print(next_chars[:3])

["\ufeff\nproject gutenberg's the adventures of ", "roject gutenberg's the adventures of she", "ect gutenberg's the adventures of sherlo"]
['s', 'r', 'c']


In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
print(x[:3])
print(y[:3])

[[[False False False ... False False  True]
  [ True False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False  True False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]]
[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fals

## Building Model

In this notebook a small recurrent neural networks is used for both simplicity and because of the training time but if you want to train a more sophisticated model you can increase the size of the network. You can also use a model pretrained on some other text like wikipedia text to both speed up the training process and get better results.

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Helper Functions
These helper functions are taken from the [official Keras text generation notebook](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py).

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Defining callbacks

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [ ]:
callbacks = [print_callback, checkpoint, reduce_lr]

## Training the model

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

Epoch 1/5
1516/1516 [==============================] - 189s 123ms/step - loss: 2.2542

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "veted, and he gave a little cry of
satis"
veted, and he gave a little cry of
satister of the starter of the sint of the stant of the stant of the can the stant of the bent had been the count to the state in the counter in the start in the saters of the man to the sent the startion of the stant of the sall be to the stoner and the care to the stoner what he was a can the started to the bention and the countly to the some to the starter of the some to the stoner man to the contre
----- diversity: 0.5
----- Generating with seed: "veted, and he gave a little cry of
satis"
veted, and he gave a little cry of
satistation which he was been a firder so the sa a to matter farish and the been the wind be a seart be in the
bent in the
counted of the can out on the bearss a stare and be the best in the tan is in the bent stentl

## Testing the model
Now that we have a trained network we can test it using a method simular to the ``on_epoch_end`` method above.

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
print(generate_text(500, 0.2))

dy between the wharf and the
house. it seemed to the comply and the companion of the shoulder the shoulder and and the corner of the state of the shoulder of the station of the was and a comply and a comprict of the state and had been a charce and a comprict of the state of the some of the companion and the compries of the part of the ble of the companion and the confiated the state and a companion of the trust of the comprict in the london and the companion. he was a crime and the state and and the companion of the shoulder and the s
